<a href="https://colab.research.google.com/github/marco-siino/gpt_networking/blob/main/Q3/Gemma_7B_Instruct_LogKnowledgeExtraction_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and load the model.

In [1]:
import os
import random
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

model_id = "/leonardo/home/userexternal/msiino00/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/18329f019fb74ca4b24f97371785268543d687d2"

#model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=dtype,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
!nvidia-smi

Wed Jun 19 07:53:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   44C    P0               75W / 462W|   5213MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Gemma 7B Instruct example usage on the local model.

In [16]:
log_sample_1 = """Given the following LOG, How many retransmissions have occurred for each dev_eui?

LOG:
time: 2019-05-20 00:12:22.271000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3843
time: 2019-05-20 00:17:25.006000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4086
time: 2019-05-20 00:19:16.051000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4098
time: 2019-05-20 00:26:39.495000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:26:45.629000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:38:24.563000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3914
time: 2019-05-20 00:40:34.091000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4093
time: 2019-05-20 00:47:19.146000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -99.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4044
time: 2019-05-20 00:49:05.496000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 142
time: 2019-05-20 01:12:21.941000, freq: 868.1, chan: 21, lsnr: 1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3844
time: 2019-05-20 01:17:24.022000, freq: 868.1, chan: 21, lsnr: 5.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4087
time: 2019-05-20 01:26:32.350000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3843
time: 2019-05-20 01:26:44.740000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3843
time: 2019-05-20 01:38:24.097000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3915
time: 2019-05-20 01:40:33.129000, freq: 868.3, chan: 22, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4094
time: 2019-05-20 01:40:39.325000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4094
time: 2019-05-20 01:47:18.235000, freq: 868.1, chan: 21, lsnr: 10.0, rssi: -104.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4045
time: 2019-05-20 01:49:04.597000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 143

"""
answer_1 = """
 dev_eui: 120 number of retransmissions: 0
 dev_eui: 121 number of retransmissions: 1
 dev_eui: 122 number of retransmissions: 0
 dev_eui: 123 number of retransmissions: 0
 dev_eui: 124 number of retransmissions: 2
 dev_eui: 126 number of retransmissions: 0
 dev_eui: 127 number of retransmissions: 0
 dev_eui: 128 number of retransmissions: 0
"""

log_sample_2 = """Given the following LOG, How many retransmissions have occurred for each dev_eui?

LOG:
time: 2019-07-03 20:16:22.628000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 511
time: 2019-07-03 20:25:06.074000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 5214
time: 2019-07-03 20:31:32.918000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 5035
time: 2019-07-03 20:32:07.259000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -89.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 5165
time: 2019-07-03 20:34:18.079000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 1263
time: 2019-07-03 20:58:58.670000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 714
time: 2019-07-03 21:01:37.582000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 5208
time: 2019-07-03 21:05:04.389000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4965
time: 2019-07-03 21:11:06.255000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4964
time: 2019-07-03 21:12:03.327000, freq: 868.5, chan: 23, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 5220
time: 2019-07-03 21:16:21.927000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 512
time: 2019-07-03 21:25:05.198000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 5215
time: 2019-07-03 21:31:32.591000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 5036
time: 2019-07-03 21:32:06.388000, freq: 868.5, chan: 23, lsnr: 14.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 5166
time: 2019-07-03 21:34:17.135000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -89.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 1264
time: 2019-07-03 21:58:57.708000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 715
time: 2019-07-03 22:01:36.652000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 5209
time: 2019-07-03 22:05:04.116000, freq: 868.1, chan: 21, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4966
time: 2019-07-03 22:11:05.489000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4965
time: 2019-07-03 22:12:02.908000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 5221

"""
answer_2 = """
 dev_eui: 120 number of retransmissions: 0
 dev_eui: 121 number of retransmissions: 0
 dev_eui: 122 number of retransmissions: 0
 dev_eui: 123 number of retransmissions: 0
 dev_eui: 124 number of retransmissions: 0
 dev_eui: 125 number of retransmissions: 0
 dev_eui: 126 number of retransmissions: 0
 dev_eui: 127 number of retransmissions: 0
 dev_eui: 128 number of retransmissions: 0
 dev_eui: 129 number of retransmissions: 0
"""

log_sample_3 = """Given the following LOG, How many retransmissions have occurred for each dev_eui?

LOG:
time: 2019-06-02 14:12:35.987000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4425
time: 2019-06-02 14:12:42.187000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4425
time: 2019-06-02 14:12:48.316000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4425
time: 2019-06-02 14:21:47.669000, freq: 868.1, chan: 21, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4182
time: 2019-06-02 14:22:00.005000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4182
time: 2019-06-02 14:35:50.505000, freq: 868.1, chan: 21, lsnr: 8.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4433
time: 2019-06-02 14:36:14.936000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4254
time: 2019-06-02 14:42:35.346000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4384
time: 2019-06-02 14:44:23.126000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 482
time: 2019-06-02 15:10:17.381000, freq: 868.3, chan: 22, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4184
time: 2019-06-02 15:12:35.049000, freq: 868.5, chan: 23, lsnr: 2.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4426
time: 2019-06-02 15:12:41.244000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4426
time: 2019-06-02 15:12:47.372000, freq: 868.3, chan: 22, lsnr: 7.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4426
time: 2019-06-02 15:21:52.916000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4183
time: 2019-06-02 15:21:59.109000, freq: 868.5, chan: 23, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4183
time: 2019-06-02 15:35:49.580000, freq: 868.1, chan: 21, lsnr: 1.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4434
time: 2019-06-02 15:36:14.559000, freq: 868.3, chan: 22, lsnr: -1.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4255
time: 2019-06-02 15:42:34.493000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4385
time: 2019-06-02 15:44:22.206000, freq: 868.3, chan: 22, lsnr: 13.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 483
time: 2019-06-02 16:10:16.159000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4185

"""
answer_3 = """
 dev_eui: 120 number of retransmissions: 0
 dev_eui: 121 number of retransmissions: 0
 dev_eui: 123 number of retransmissions: 0
 dev_eui: 124 number of retransmissions: 2
 dev_eui: 126 number of retransmissions: 4
 dev_eui: 127 number of retransmissions: 0
 dev_eui: 128 number of retransmissions: 0
"""

log_test = """

Given the following LOG, How many retransmissions have occurred for each dev_eui?

LOG:
time: 2019-06-30 16:12:38.597000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 25, FCnt: 5140
time: 2019-06-30 16:17:04.515000, freq: 868.1, chan: 21, lsnr: 4.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 432
time: 2019-06-30 16:26:11.191000, freq: 868.1, chan: 21, lsnr: 7.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 5135
time: 2019-06-30 16:32:01.241000, freq: 868.1, chan: 21, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4956
time: 2019-06-30 16:33:12.403000, freq: 868.5, chan: 23, lsnr: 14.0, rssi: -83.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 5086
time: 2019-06-30 16:35:24.809000, freq: 868.3, chan: 22, lsnr: 13.0, rssi: -98.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 1184
time: 2019-06-30 17:00:08.061000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -83.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 635
time: 2019-06-30 17:02:43.811000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 5129
time: 2019-06-30 17:05:32.056000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4886
time: 2019-06-30 17:12:09.892000, freq: 868.3, chan: 22, lsnr: -2.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 25, FCnt: 4885
time: 2019-06-30 17:12:38.300000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 5141
time: 2019-06-30 17:17:04.149000, freq: 868.5, chan: 23, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 23, FCnt: 433
time: 2019-06-30 17:26:10.316000, freq: 868.1, chan: 21, lsnr: 4.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 5136
time: 2019-06-30 17:32:01.976000, freq: 868.1, chan: 21, lsnr: -4.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4957
time: 2019-06-30 17:33:11.647000, freq: 868.5, chan: 23, lsnr: 14.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 5087
time: 2019-06-30 17:35:23.894000, freq: 868.3, chan: 22, lsnr: 13.0, rssi: -89.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 1185
time: 2019-06-30 18:00:07.100000, freq: 868.1, chan: 21, lsnr: 15.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 636
time: 2019-06-30 18:02:42.898000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 5130
time: 2019-06-30 18:05:31.768000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 25, FCnt: 4887
time: 2019-06-30 18:12:08.965000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4886

"""


#chat = [
#    { "role": "user", "content": "Write a hello world program" },
#]

chat = [
    { "role": "user", "content": log_sample_1 },
    { "role": "model", "content": answer_1 },
    { "role": "user", "content": log_sample_2 },
    { "role": "model", "content": answer_2 },
    { "role": "user", "content": log_sample_3 },
    { "role": "model", "content": answer_3 },
    { "role": "user", "content": log_test },
]


prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1500)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
Given the following LOG, How many retransmissions have occurred for each dev_eui?

LOG:
time: 2019-05-20 00:12:22.271000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3843
time: 2019-05-20 00:17:25.006000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4086
time: 2019-05-20 00:19:16.051000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4098
time: 2019-05-20 00:26:39.495000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:26:45.629000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:38:24.563000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3914
time: 2

# Generic Gemma 7B usage.

In [ ]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/leonardo/home/userexternal/msiino00/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/18329f019fb74ca4b24f97371785268543d687d2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    revision="float16",
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, )
print(tokenizer.decode(outputs[0]))


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/leonardo/home/userexternal/msiino00/clean-updated-env/lib/python3.10/site-packages/transformers/generation/utils.py:1244: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

In the realm of data, a tale unfolds,


# Organize the dataset.

In [ ]:
!unzip prompt_Q3-2024-06-13.zip

Archive:  prompt_Q3-2024-06-13.zip
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1076.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1074.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1072.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1088.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1090.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1082.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1070.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1078.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1086.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1084.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1080.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1036.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1044.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=998.txt  
  inflating: prompt_Q3/prompt_n_hours=2_skip_hours=1030.txt  
  inflating: prompt_Q3/prompt_n_hour

In [4]:
folder_path="log_knowledge_extraction/prompt_Q3"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [58]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                log_sample ="Given the following LOG, How many retransmissions have occurred for each dev_eui?\n\nLOG:\n"

                parts = file_contents.split('QUESTION:')
                log_sample += parts[0]
                log_sample += "ANSWER:"

                parts = file_contents.split('ANSWER:')
                answer = parts[1]

                question = { "role": "user", "content": log_sample }
                answer = { "role": "model", "content": answer }

                return question,answer


In [55]:
file_contents = """
time: 2019-05-20 00:12:22.271000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3843
time: 2019-05-20 00:17:25.006000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4086
time: 2019-05-20 00:19:16.051000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4098
time: 2019-05-20 00:26:39.495000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:26:45.629000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:38:24.563000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3914
time: 2019-05-20 00:40:34.091000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4093
time: 2019-05-20 00:47:19.146000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -99.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4044
time: 2019-05-20 00:49:05.496000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -88.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 142
time: 2019-05-20 01:12:21.941000, freq: 868.1, chan: 21, lsnr: 1.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3844
time: 2019-05-20 01:17:24.022000, freq: 868.1, chan: 21, lsnr: 5.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4087
time: 2019-05-20 01:26:32.350000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3843
time: 2019-05-20 01:26:44.740000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3843
time: 2019-05-20 01:38:24.097000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3915
time: 2019-05-20 01:40:33.129000, freq: 868.3, chan: 22, lsnr: 6.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4094
time: 2019-05-20 01:40:39.325000, freq: 868.1, chan: 21, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 25, FCnt: 4094
time: 2019-05-20 01:47:18.235000, freq: 868.1, chan: 21, lsnr: 10.0, rssi: -104.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4045
time: 2019-05-20 01:49:04.597000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -84.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 143


QUESTION: How many retransmissions have occurred for each dev_eui in 2 hours?

ANSWER:
 dev_eui: 120 number of retransmissions: 0
 dev_eui: 121 number of retransmissions: 1
 dev_eui: 122 number of retransmissions: 0
 dev_eui: 123 number of retransmissions: 0
 dev_eui: 124 number of retransmissions: 2
 dev_eui: 126 number of retransmissions: 0
 dev_eui: 127 number of retransmissions: 0
 dev_eui: 128 number of retransmissions: 0

"""

question, answer = preprocess_sample(file_contents)

print(question['content'])

Given the following LOG, How many retransmissions have occurred for each dev_eui?

time: 2019-05-20 00:12:22.271000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3843
time: 2019-05-20 00:17:25.006000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4086
time: 2019-05-20 00:19:16.051000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4098
time: 2019-05-20 00:26:39.495000, freq: 868.5, chan: 23, lsnr: -9.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:26:45.629000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3842
time: 2019-05-20 00:38:24.563000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3914
time: 2019-05-20 00:40:34.091000, fre

# Create the few-shots samples.

In [59]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = []

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                question, answer = preprocess_sample(file_contents)

                few_shots_set.append(question)
                few_shots_set.append(answer)


                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break
        test_set.remove(filename)

    return few_shots_set

In [39]:
# Specify the number of files to read
n = 3  # Example: Read the first 5 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=42.txt
prompt_n_hours=2_skip_hours=480.txt
prompt_n_hours=2_skip_hours=1016.txt
[{'role': 'user', 'content': 'Given the following LOG, How many retransmissions have occurred for each dev_eui?\n                time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130\ntime: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142\ntime: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886\ntime: 2019-05-21 18:38:0

# Chat with the model to get the results.

In [63]:
# Define the directory path containing the results.
folder_name = "log_knowledge_extraction/gemma_7b_Q3_results/"
directory_path = os.path.join(os.getcwd(), folder_name)

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        if(os.path.isfile(directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Create the directory if it doesn't exist
            if not os.path.exists(directory_path):
                os.makedirs(directory_path)

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()
                chat = ""
                # Specify the number of files to read
                n = 5  # Example: Read the first 5 files
                # Call the function to read text files in the folder and concatenate their contents
                few_shots_samples = create_few_shots_set(folder_path, n)

                chat = few_shots_samples
                question,anwer = preprocess_sample(file_contents)

                chat.append(question)

                prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
                print("****.  ECCCO IL PROMPT !!!! **** ")
                print(prompt)
                print("\n\n\n\n")

                inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
                outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=1000)
                result = tokenizer.decode(outputs[0])
                print("****.  ECCCO LA RISPOSTA !!!! **** ")
                print(result)

                # Create the directory if it doesn't exist
                if not os.path.exists(directory_path):
                    os.makedirs(directory_path)

                # Define the file path
                file_path = os.path.join(directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)
        break

prompt_n_hours=2_skip_hours=36.txt
Already done!


prompt_n_hours=2_skip_hours=1082.txt
Already done!


prompt_n_hours=2_skip_hours=748.txt
Already done!


prompt_n_hours=2_skip_hours=100.txt
Already done!


prompt_n_hours=2_skip_hours=348.txt
Already done!


prompt_n_hours=2_skip_hours=140.txt
prompt_n_hours=2_skip_hours=42.txt
prompt_n_hours=2_skip_hours=480.txt
prompt_n_hours=2_skip_hours=1016.txt
prompt_n_hours=2_skip_hours=786.txt
prompt_n_hours=2_skip_hours=348.txt
****.  ECCCO IL PROMPT !!!! **** 
<bos><start_of_turn>user
Given the following LOG, How many retransmissions have occurred for each dev_eui?

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, S

In [ ]:
!zip -r mistral_7b_Q3_results.zip mistral_7b_Q3_results/

  adding: mistral_7b_Q3_results/ (stored 0%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=478.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=718.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=310.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=634.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=464.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=760.txt (deflated 65%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=324.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=828.txt (deflated 80%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=144.txt (deflated 78%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=582.txt (deflated 79%)
  adding: mistral_7b_Q3_results/prompt_n_hours=2_skip_hours=304.txt (deflated 68%)
  adding: mistral_7b_Q3_results/prompt_n_h

# Given the same few-shot samples ask to Mistral to generate the Python code to perform the task.

In [65]:
prompt ="""
Given the following string of a LOG, write a Python function to OUTPUT (as in the example below) how many retransmissions have occurred for each dev_eui.
LOG:
time: 2019-06-21 18:14:01.096000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4917
time: 2019-06-21 18:15:17.743000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4661
time: 2019-06-21 18:15:23.937000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4661
time: 2019-06-21 18:19:03.299000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 25, FCnt: 209
time: 2019-06-21 18:29:13.872000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4912
time: 2019-06-21 18:33:10.313000, freq: 868.1, chan: 21, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4733
time: 2019-06-21 18:36:02.130000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -86.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 23, FCnt: 4863
time: 2019-06-21 18:38:14.847000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -91.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 961
time: 2019-06-21 19:03:22.842000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -81.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 412
time: 2019-06-21 19:05:50.163000, freq: 868.5, chan: 23, lsnr: 6.0, rssi: -107.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4906
time: 2019-06-21 19:07:06.216000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4663
time: 2019-06-21 19:07:12.413000, freq: 868.1, chan: 21, lsnr: -6.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4663
time: 2019-06-21 19:07:18.547000, freq: 868.5, chan: 23, lsnr: -3.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4663
time: 2019-06-21 19:14:00.626000, freq: 868.1, chan: 21, lsnr: -8.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4918
time: 2019-06-21 19:15:16.953000, freq: 868.1, chan: 21, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4662
time: 2019-06-21 19:15:23.151000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4662
time: 2019-06-21 19:19:02.849000, freq: 868.1, chan: 21, lsnr: 9.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 25, FCnt: 210
time: 2019-06-21 19:19:08.978000, freq: 868.5, chan: 23, lsnr: 9.0, rssi: -105.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 25, FCnt: 210
time: 2019-06-21 19:19:15.175000, freq: 868.5, chan: 23, lsnr: 7.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 25, FCnt: 210
time: 2019-06-21 19:29:13.033000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -111.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4913
time: 2019-06-21 19:33:10.049000, freq: 868.3, chan: 22, lsnr: -4.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 4734
time: 2019-06-21 19:36:01.237000, freq: 868.3, chan: 22, lsnr: 15.0, rssi: -85.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4864
time: 2019-06-21 19:38:14, freq: 868.3, chan: 22, lsnr: 12.0, rssi: -94.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 962
time: 2019-06-21 20:03:21.908000, freq: 868.3, chan: 22, lsnr: 14.0, rssi: -80.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 125, size: 23, FCnt: 413
time: 2019-06-21 20:05:49.269000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4907
time: 2019-06-21 20:07:05.913000, freq: 868.1, chan: 21, lsnr: -3.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4664
time: 2019-06-21 20:07:12.039000, freq: 868.5, chan: 23, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 4664


OUTPUT:
 dev_eui: 120 number of retransmissions: 0
 dev_eui: 121 number of retransmissions: 0
 dev_eui: 122 number of retransmissions: 0
 dev_eui: 123 number of retransmissions: 0
 dev_eui: 124 number of retransmissions: 2
 dev_eui: 125 number of retransmissions: 0
 dev_eui: 126 number of retransmissions: 0
 dev_eui: 127 number of retransmissions: 0
 dev_eui: 128 number of retransmissions: 3
 dev_eui: 129 number of retransmissions: 2


"""

In [68]:
chat_new = [
    { "role": "user", "content": prompt }
]


prompt = tokenizer.apply_chat_template(chat_new, tokenize=False, add_generation_prompt=True)

print(prompt)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=5000)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
<bos><start_of_turn>user
<bos><start_of_turn>user
Given the following string of a LOG, write a Python function to OUTPUT (as in the example below) how many retransmissions have occurred for each dev_eui.
LOG:
time: 2019-06-21 18:14:01.096000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4917
time: 2019-06-21 18:15:17.743000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4661
time: 2019-06-21 18:15:23.937000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4661
time: 2019-06-21 18:19:03.299000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 25, FCnt: 209
time: 2019-06-21 18:29:13.872000, freq: 868.5, chan: 23, lsnr: 3.0, rssi: -110.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4912
time: 2019-06-21 18:33:10.313000, 

In [69]:
import re

# Define the function to count the number of retransmissions for each dev_eui
def count_retransmissions(log):
    # Create a dictionary to store the number of retransmissions for each dev_eui
    retransmissions_by_dev_eui = {}

    # Iterate over the log entries
    for entry in log.splitlines():
        # Extract the dev_eui and the number of retransmissions
        dev_eui = entry["dev_eui"]
        num_retransmissions = int(entry["retransmissions"])

        # If the dev_eui is not already in the dictionary, add it with a count of 0
        if dev_eui not in retransmissions_by_dev_eui:
            retransmissions_by_dev_eui[dev_eui] = 0

        # Increment the number of retransmissions for the dev_eui
        retransmissions_by_dev_eui[dev_eui] += num_retransmissions

    # Print the number of retransmissions for each dev_eui
    for dev_eui, num_retransmissions in retransmissions_by_dev_eui.items():
        print(dev_eui + " number of retransmissions: ", num_retransmissions)


# Example usage
log = """
time: 2019-06-21 18:14:01.096000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4917
time: 2019-06-21 18:15:17.743000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4661
time: 2019-06-21 18:15:23.937000, freq: 868.3, chan: 22, lsnr: -7.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 4661
time: 2019-06-21 18:19:03.299000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 129, size: 25, FCnt: 209
"""

count_retransmissions(log)

TypeError: string indices must be integers

In [ ]:
for filename in test_set:
        file_path = folder_name+filename
        with open(os.path.join(folder_path, filename), 'r') as file:
          file_contents = file.read()
          find_duplicate_frames(log_data)
        break

ValueError: not enough values to unpack (expected 2, got 0)

## Let's run the code.

In [ ]:
!rm -r 'mistral_7b_source_code_results/'

In [ ]:
# Define the directory path containing the results.
folder_name = 'mistral_7b_Q2_source_code_results/'
#directory_path = os.path.join(os.getcwd(), folder_name)
# Create the directory if it doesn't exist
if not os.path.exists(folder_name):
  os.makedirs(folder_name)

# Iterate through all the test_files
for filename in test_set:
        file_path = folder_name+filename
        print(filename)
        print(file_path)
        if(os.path.isfile(file_path)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Remove the ANSWER in the original log file.
                file_contents = file_contents.split("ANSWER", 1)

                result = process_log(file_contents[0])

                print(result+"\n\n")

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=268.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=268.txt
For DEVICE dev_eui:120 all received frames are 4.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=924.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=924.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 1.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 1.
For DEVICE dev_eui:125 all received fra

# Here the function to run the evaluation at file-level and at global-level.

In [ ]:
def evaluate_results (dataset_path,result_path):
  dataset_corpus = ""
  results_corpus = ""
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      dataset_corpus += text2
      results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  reference = results_corpus.split()
  candidate = dataset_corpus.split()

  # Compute BLEU score
  smoothing_function = SmoothingFunction().method1
  bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

  # Compute ROUGE score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  rouge_scores = scorer.score(text1, text2)

  # Print the results
  print(f"BLEU score: {bleu_score}")
  print(f"ROUGE-1: {rouge_scores['rouge1']}")
  print(f"ROUGE-L: {rouge_scores['rougeL']}")

## Let's evaluate the results of Mistral.

In [ ]:
dataset_path = "prompt_Q2/"
result_path = "mistral_7b_Q2_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
Sample:  prompt_n_hours=2_skip_hours=318.txt
dev_eui: 124 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)
ROUGE-L: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)



Sample:  prompt_n_hours=2_skip_hours=390.txt
dev_eui: 124 
 dev_eui: 126 
 dev_eui: 128


dev_eui: 123, dev_eui: 128, dev_eui: 124, dev_eui: 126
BLEU score: 0.07356556000975763
ROUGE-1: Score(precision=1.0, recall=0.75, fmeasure=0.8571428571428571)
ROUGE-L: Score(precision=0.6666666666666666, recall=0.5, fmeasure=0.5714285714285715)



Sample:  prompt_n_hours=2_skip_hours=710.txt
dev_eui: 120 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=0.8333333333333334, recall=0.5555555555555556, fmeasure=0.6666666666666667)
ROUGE-L: Score(precision=0.8333333333333334, recall=0.555555555555555

## Let's evaluate the results of the source code generated by Mistral.

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "mistral_7b_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.
BLEU score: 1.0
ROUGE-1: Score(precision=1.0, recall=1.0, fmeasure=1.0)
ROUGE-L: Score(precision=1.0, recall=1.0, fmeasure=1.0)



Sample:  prompt_n_hours=2_skip_hours=362.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 